In [1]:
import fastf1
import fastf1.api
import fastf1.plotting

import pandas as pd
import numpy as np
import os  
from datetime import timedelta
import csv
import seaborn as sns
from fastf1.ergast import Ergast

ergast = Ergast()

year = 2025
race_number = 7
race_session = 'Q'
Lap_Number = 31
team='Ferrari'
quali = 'Q1'
#year = int(input("Year ? "))
#race_number = int(input("Race Number ? (1-24) "))
#race_session = input('Session ?  (S, R) ')

session= fastf1.get_session(year, race_number, race_session)
session.load()

pit = ergast.get_pit_stops(season = year, round = race_number )
teams = fastf1.plotting.list_team_names(session)
team_drivers = fastf1.plotting.get_driver_abbreviations_by_team(team, session=session)
event_name = session.event.EventName

/home/kurios/Documents/f1_analysis/f1_analysis_env/lib/python3.10/site-packages/fastf1/api.py:32: UserWarning: `fastf1.api` will be considered private in future releases and potentially be removed or changed!
  warnings.warn("`fastf1.api` will be considered private in future releases and "
req         WARNING 	DEFAULT CACHE ENABLED! (30.47 GB) /home/kurios/.cache/fastf1
core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using

In [12]:
def highlight(s, min_max):
    if min_max == 'min':
        is_min_max = s == s.min()
    if min_max == 'max':
        is_min_max = s == s.max()

    return ['color: green' if v else 'color: orange' for v in is_min_max]

def background_color_df(s):
    colors= []
    for time_and_compound in s:
        if ' SOFT' in time_and_compound:
            colors.append('background-color:red')
        if ' MEDIUM' in time_and_compound:
            colors.append('background-color:yellow')
        if ' HARD' in time_and_compound:
            colors.append('background-color:white')
        if ' INTERMEDIATE' in time_and_compound:
            colors.append('background-color:green')
        if ' WET' in time_and_compound:
            colors.append('background-color:blue')
        if ' SUPERSOFT' in time_and_compound:
            colors.append('background-color:purple')
        if ' ULTRASOFT' in time_and_compound:
            colors.append('background-color:orangered')
        if ' HYPERSOFT' in time_and_compound:
            colors.append('background-color:pink')
        if ' SUPERHARD' in time_and_compound:
            colors.append('background-color:orange')
        if 'No data' in time_and_compound:
            colors.append('background-color:grey')
    return colors

def highlight_driver(s):
    drivers = []
    for driver_name in s:
        driver_color = fastf1.plotting.get_driver_color(driver_name[0:3], session)
        drivers.append(f'color: {driver_color}')
    return drivers

def color_df(s, color):
    colors= []
    for driver_name in s:
        colors.append(f'color:{color}')
    return colors

def is_personal_best_min(s, column):
    best_colors = []
    min_values = {}
    driver_ids = list(session.drivers)

    for driver_id in driver_ids:
        min_values[driver_id] = session.laps.pick_drivers(driver_id).pick_laps(range(0, Lap_Number + 1))[column].min()

    all_driver_laps = session.laps.pick_drivers(session.drivers).pick_laps(range(0, Lap_Number + 1))
    if not all_driver_laps.empty:
        overall_min = all_driver_laps[column].min()
    else:
        overall_min = 'inf'

    for column_value in s:
        color = 'color:orange'
        for driver in session.drivers:
            if column_value  == overall_min:
                color = 'color:purple'
            if column_value == min_values.get(driver):
                color = 'color:green'

        best_colors.append(color)

    return best_colors

def is_personal_best_max(s, column):
    best_colors = []
    max_values = {}
    driver_ids = list(session.drivers)

    for driver_id in driver_ids:
        max_values[driver_id] = session.laps.pick_drivers(driver_id).pick_laps(range(0, Lap_Number + 1))[column].max()

    all_driver_laps = session.laps.pick_drivers(session.drivers).pick_laps(range(0, Lap_Number + 1))
    if not all_driver_laps.empty:
        overall_max = all_driver_laps[column].max()
    else:
        overall_max = 0
    
    for column_value in s:
        color = 'color:orange'
        for driver in session.drivers:
            if column_value == overall_max:
                color = 'color:purple'
            elif column_value == max_values.get(driver):
                color = 'color:green'

        best_colors.append(color)

    return best_colors

def flag_color_row(s):
    flag_colors=[]
    for flag_color in s:
        match flag_color:
            case flag_color if 'GREEN' in flag_color:
                flag_colors.append('color:green')
            case flag_color if 'YELLOW' in flag_color:
                flag_colors.append('color:yellow')
            case flag_color if 'DOUBLE YELLOW' in flag_color:
                flag_colors.append('color:orange')
            case flag_color if 'RED' in flag_color:
                flag_colors.append('color:red')
            case flag_color if 'BLUE' in flag_color:
                flag_colors.append('color:blue')
            case flag_color if 'CLEAR' in flag_color:
                flag_colors.append('color:white')
            case flag_color if 'BLACK' in flag_color:
                flag_colors.append('color:dark-grey')
            case _:
                flag_colors.append('color:grey')
    return flag_colors

def get_wind_direction_cat(WindDirection):
    normalized_direction = WindDirection % 360
    if normalized_direction < 0:
        normalized_direction += 360
    match normalized_direction:
        case _ if (normalized_direction >= 348.75 or normalized_direction < 11.25):
            return 'N'
        case _ if (normalized_direction >= 11.25 and normalized_direction < 33.75):
            return 'NNE'
        case _ if (normalized_direction >= 33.75 and normalized_direction < 56.25):
            return 'NE'
        case _ if (normalized_direction >= 56.25 and normalized_direction < 78.75):
            return 'ENE'
        case _ if (normalized_direction >= 78.75 and normalized_direction < 101.25):
            return 'E'
        case _ if (normalized_direction >= 101.25 and normalized_direction < 123.75):
            return 'ESE'
        case _ if (normalized_direction >= 123.75 and normalized_direction < 146.25):
            return 'SE'
        case _ if (normalized_direction >= 146.25 and normalized_direction < 168.75):
            return 'SSE'
        case _ if (normalized_direction >= 168.75 and normalized_direction < 191.25):
            return 'S'
        case _ if (normalized_direction >= 191.25 and normalized_direction < 213.75):
            return 'SSW'
        case _ if (normalized_direction >= 213.75 and normalized_direction < 236.25):
            return 'SW'
        case _ if (normalized_direction >= 236.25 and normalized_direction < 258.75):
            return 'WSW'
        case _ if (normalized_direction >= 258.75 and normalized_direction < 281.25):
            return 'W'
        case _ if (normalized_direction >= 281.25 and normalized_direction < 303.75):
            return 'WNW'
        case _ if (normalized_direction >= 303.75 and normalized_direction < 326.25):
            return 'NW'
        case _ if (normalized_direction >= 326.25 and normalized_direction < 348.75):
            return 'NNW'
        case _:
            return 'Invalid'

tyres={
    'SOFT':'🔴',
    'MEDIUM':'🟡',
    'HARD':'⚪',
    'INTERMEDIATE':'🟢',
    'WET':'🔵',
    'SUPERSOFT': '🟣',
    'ULTRASOFT': '🟠',
    'HYPERSOFT': '🩷',
    'SUPERHARD': '🍊',
}


In [13]:
Lap_Number = 40
driver_data_last_laps = []

driver = team_drivers[0]
driver_lap = session.laps.pick_drivers(driver)
max_lap = int(max(driver_lap.LapNumber))

if Lap_Number<10:
    lap_range =range(0, min(Lap_Number, max_lap))
else :
    lap_range = range(min(Lap_Number-10, max_lap - 10), min(Lap_Number, max_lap))

for lap in lap_range:
    driver_data_cols = ['Time', 'Lap', 'Sector1', 'I1', 'Sector2', 'I2', 'Sector3', 'FL', 'LapTime', 'ST']
    
    try:

        driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[lap], session)
        driver_data = [driver_lap.Time.iloc[lap], driver_lap.LapNumber.iloc[lap], driver_lap.Sector1Time.iloc[lap], driver_lap.SpeedI1.iloc[lap], driver_lap.Sector2Time.iloc[lap], driver_lap.SpeedI2.iloc[lap], driver_lap.Sector3Time.iloc[lap], driver_lap.SpeedFL.iloc[lap], driver_lap.LapTime.iloc[lap], driver_lap.SpeedST.iloc[lap]]
    except:
        continue
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    driver_data_last_laps.append(driver_data_series)

driver_data_last_laps_df = pd.DataFrame(driver_data_last_laps)

styled_df = driver_data_last_laps_df.style

styled_df = styled_df.apply(highlight, subset=['Sector1'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['I1'], min_max='max')
styled_df = styled_df.apply(highlight, subset=['Sector2'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['I2'], min_max='max')
styled_df = styled_df.apply(highlight, subset=['Sector3'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['FL'], min_max='max')
styled_df = styled_df.apply(highlight, subset=['LapTime'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['ST'], min_max='max')

formatters = {
    'Time': lambda x: str(x)[7:-3] if pd.notnull(x) else 'Ø',
    'Lap': lambda x: int(x) if pd.notnull(x) else 'Ø',
    'Sector1': lambda x: str(x)[13:-3] if pd.notnull(x) else 'Ø',
    'I1': lambda x: int(x) if pd.notnull(x) else 'Ø',
    'Sector2': lambda x: str(x)[13:-3] if pd.notnull(x) else 'Ø',
    'I2': lambda x: int(x) if pd.notnull(x) else 'Ø',
    'Sector3': lambda x: str(x)[13:-3] if pd.notnull(x) else 'Ø',
    'FL': lambda x: int(x) if pd.notnull(x) else 'Ø',
    'LapTime': lambda x: str(x)[11:-3] if pd.notnull(x) else 'Ø',
    'ST': lambda x: int(x) if pd.notnull(x) else 'Ø',
}

final_formatters = {k: v for k, v in formatters.items() if k in driver_data_last_laps_df.columns}
styled_df = styled_df.format(final_formatters)
styled_df

,Time,Lap,Sector1,I1,Sector2,I2,Sector3,FL,LapTime,ST
0,01:02:42.521,12,19.632,193,39.336,175,Ø,Ø,Ø,225
1,01:04:24.265,13,Ø,281,41.274,214,26.112,287,Ø,285
2,01:05:28.999,14,16.391,318,28.867,247,19.476,287,1:04.734,317
3,01:06:54.209,15,19.753,238,36.056,196,29.401,Ø,1:25.210,227
4,01:18:03.020,16,Ø,288,41.652,137,21.784,286,Ø,296
5,01:19:07.512,17,16.371,315,28.763,249,19.358,285,1:04.492,321
6,01:20:25.963,18,18.638,288,33.081,223,26.732,Ø,1:18.451,263
7,01:24:31.875,19,Ø,288,34.186,144,26.152,288,Ø,298
8,01:25:36.376,20,16.326,315,28.725,251,19.450,285,1:04.501,318
9,01:27:14.083,21,21.282,208,39.596,186,36.829,Ø,1:37.707,185


EACH DRIVER

In [14]:
def get_wind_direction_cat(WindDirection):
    normalized_direction = WindDirection % 360
    if normalized_direction < 0:
        normalized_direction += 360
    match normalized_direction:
        case _ if (normalized_direction >= 348.75 or normalized_direction < 11.25):
            return 'N'
        case _ if (normalized_direction >= 11.25 and normalized_direction < 33.75):
            return 'NNE'
        case _ if (normalized_direction >= 33.75 and normalized_direction < 56.25):
            return 'NE'
        case _ if (normalized_direction >= 56.25 and normalized_direction < 78.75):
            return 'ENE'
        case _ if (normalized_direction >= 78.75 and normalized_direction < 101.25):
            return 'E'
        case _ if (normalized_direction >= 101.25 and normalized_direction < 123.75):
            return 'ESE'
        case _ if (normalized_direction >= 123.75 and normalized_direction < 146.25):
            return 'SE'
        case _ if (normalized_direction >= 146.25 and normalized_direction < 168.75):
            return 'SSE'
        case _ if (normalized_direction >= 168.75 and normalized_direction < 191.25):
            return 'S'
        case _ if (normalized_direction >= 191.25 and normalized_direction < 213.75):
            return 'SSW'
        case _ if (normalized_direction >= 213.75 and normalized_direction < 236.25):
            return 'SW'
        case _ if (normalized_direction >= 236.25 and normalized_direction < 258.75):
            return 'WSW'
        case _ if (normalized_direction >= 258.75 and normalized_direction < 281.25):
            return 'W'
        case _ if (normalized_direction >= 281.25 and normalized_direction < 303.75):
            return 'WNW'
        case _ if (normalized_direction >= 303.75 and normalized_direction < 326.25):
            return 'NW'
        case _ if (normalized_direction >= 326.25 and normalized_direction < 348.75):
            return 'NNW'
        case _:
            return 'Invalid'

Lap_Number=10
minute=20

weather_driver_lap = session.laps.get_weather_data()
weather_driver_lap = weather_driver_lap.drop_duplicates()
weather_driver_lap = weather_driver_lap.sort_values(['Time'])
weather_driver_lap['Direction'] = weather_driver_lap['WindDirection'].apply(get_wind_direction_cat)
#weather_driver_lap['Time_sec'] = weather_driver_lap.Time.dt.total_seconds()
#print((weather_driver_lap.Time_sec.iloc[0] + 600))
weather_evolution = [weather_driver_lap.iloc[0],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*2],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*3],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*4],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*5],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*6],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*7],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*8],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)/10)*9],
                     weather_driver_lap.iloc[int(len(weather_driver_lap)-1)]]
weather_evolution_df = pd.DataFrame(weather_evolution)
weather_evolution_df.Time = weather_evolution_df.Time.astype('str').str[7:-3]
weather_evolution_df.Humidity = weather_evolution_df.Humidity.astype('int')
#weather_evolution_df = weather_evolution_df.drop(columns=['Time_sec'])
weather_evolution_df

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Direction
18,00:18:37.054,27.6,35,945.3,False,47.8,259,1.1,W
22,00:22:37.057,27.5,34,945.4,False,48.5,195,0.6,SSW
26,00:26:37.067,27.8,33,945.3,False,48.3,253,1.0,WSW
30,00:30:37.070,27.8,33,945.3,False,47.8,94,1.5,E
34,00:34:37.060,27.5,34,945.3,False,47.2,0,1.2,N
44,00:44:37.080,28.0,34,945.4,False,47.2,241,1.7,WSW
48,00:48:37.076,28.1,34,945.4,False,47.2,240,1.4,WSW
61,01:01:37.103,28.2,33,945.4,False,47.4,235,1.7,SW
65,01:05:37.102,28.0,34,945.3,False,47.1,128,1.0,SE
78,01:18:37.106,28.0,34,945.3,False,47.1,176,1.4,S


CURENT STATUS

In [ ]:

def is_personal_best(s, column):
    best= []
    for column_value in s:
        try:
            if column_value == session.laps.pick_drivers(81).pick_laps(range(0, Lap_Number+1))[column].min():
                best.append(f'color:purple')
            elif column_value == session.laps.pick_drivers(4).pick_laps(range(0, Lap_Number+1))[column].min():
                best.append(f'color:green')
            elif column_value == session.laps.pick_drivers(44).pick_laps(range(0, Lap_Number+1))[column].min():
                best.append(f'color:green')
            elif column_value == session.laps.pick_drivers(16).pick_laps(range(0, Lap_Number+1))[column].min():
                best.append(f'color:green')
            else:
                best.append(f'color:orange')
            if column_value < session.laps.pick_drivers(drivers_data_df.Driver).pick_laps(range(0, Lap_Number+1))[column].min():
                best.append(f'color:purple')
        except:
            best.append(f'color:orange')
    return best

In [1]:
import fastf1
import fastf1.api
import fastf1.plotting

import pandas as pd
import numpy as np
import os  
from datetime import timedelta
import csv
import seaborn as sns
from fastf1.ergast import Ergast

ergast = Ergast()

year = 2025
race_number = 11
race_session = 'FP1'
Lap_Number = 31
team='Ferrari'
#year = int(input("Year ? "))
#race_number = int(input("Race Number ? (1-24) "))
#race_session = input('Session ?  (S, R) ')

session= fastf1.get_session(year, race_number, race_session)
session.load()
if race_session == 'Q':
    q1, q2, q3 = session.laps.split_qualifying_sessions()
    is_nat = np.isnat(q1['LapTime'])
    q1 = q1[~is_nat]
    is_nat = np.isnat(q2['LapTime'])
    q2 = q2[~is_nat]
    is_nat = np.isnat(q3['LapTime'])
    q3 = q3[~is_nat]

pit = ergast.get_pit_stops(season = year, round = race_number )
teams = fastf1.plotting.list_team_names(session)
team_drivers = fastf1.plotting.get_driver_abbreviations_by_team(team, session=session)
event_name = session.event.EventName

/home/kurios/Documents/f1_analysis/f1_analysis_env/lib/python3.10/site-packages/fastf1/api.py:32: UserWarning: `fastf1.api` will be considered private in future releases and potentially be removed or changed!
  warnings.warn("`fastf1.api` will be considered private in future releases and "
req         WARNING 	DEFAULT CACHE ENABLED! (30.47 GB) /home/kurios/.cache/fastf1
core           INFO 	Loading data for Austrian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cache

In [ ]:
session= fastf1.get_session(2025, 2, 'R')
session.load()
print(session.event.EventFormat)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', 

conventional


In [24]:
Lap_Number=10

def is_personal_best(s, column):
    best_colors = []

    min_values = {}
    driver_ids = list(session.drivers)

    for driver_id in driver_ids:
        try:
            min_values[driver_id] = session.laps.pick_drivers(driver_id).pick_laps(range(0, Lap_Number + 1))[column].min()
        except Exception:
            min_values[driver_id] = float('inf')

    overall_min = float('inf')
    try:
        all_driver_laps = session.laps.pick_drivers(session.drivers).pick_laps(range(0, Lap_Number + 1))
        if not all_driver_laps.empty:
            overall_min = all_driver_laps[column].min()
    except Exception:
        overall_min = float('inf')

    for column_value in s:
        color = 'color:orange'
        for driver in session.drivers:
            if column_value == session.laps.pick_drivers(81).pick_laps(range(0, Lap_Number+1))[column].min():
                color = 'color:purple'
            if column_value == min_values.get(driver):
                color = 'color:green'

        if column_value < overall_min:
            color = 'color:purple'

        best_colors.append(color)

    return best_colors

def color_df(s, color):
    colors= []
    for driver_name in s:
        colors.append(f'color:{color}')
    return colors

def highlight_compound(s):
    colors = []
    for compound_value in s:
        if compound_value == 'SOFT':
            colors.append('color: red')
        elif compound_value == 'MEDIUM':
            colors.append('color: yellow')
        elif compound_value == 'HARD':
            colors.append('color: white')
        elif compound_value == 'INTERMEDIATE':
            colors.append('color: green')
        elif compound_value == 'WET':
            colors.append('color: blue')
        else:
            colors.append('color: grey')
    return colors

def highlight_driver(s):
    drivers = []
    for driver_name in s:
        driver_name = driver_name.split(' ')[0]
        driver_color = fastf1.plotting.get_driver_color(driver_name, session)
        drivers.append(f'color: {driver_color}')
    return drivers


def is_personal_best_min(s, column):
    best_colors = []
    min_values = {}
    driver_ids = list(session.drivers)

    for driver_id in driver_ids:
        min_values[driver_id] = session.laps.pick_drivers(driver_id).pick_laps(range(0, Lap_Number + 1))[column].min()

    all_driver_laps = session.laps.pick_drivers(session.drivers).pick_laps(range(0, Lap_Number + 1))
    if not all_driver_laps.empty:
        overall_min = all_driver_laps[column].min()
    else:
        overall_min = 'inf'

    for column_value in s:
        color = 'color:orange'
        for driver in session.drivers:
            if column_value  == overall_min:
                color = 'color:purple'
            if column_value == min_values.get(driver):
                color = 'color:green'

        best_colors.append(color)

    return best_colors

def is_personal_best_max(s, column):
    best_colors = []
    max_values = {}
    driver_ids = list(session.drivers)

    for driver_id in driver_ids:
        max_values[driver_id] = session.laps.pick_drivers(driver_id).pick_laps(range(0, Lap_Number + 1))[column].max()

    all_driver_laps = session.laps.pick_drivers(session.drivers).pick_laps(range(0, Lap_Number + 1))
    if not all_driver_laps.empty:
        overall_max = all_driver_laps[column].max()
    else:
        overall_max = 0
    
    for column_value in s:
        color = 'color:orange'
        for driver in session.drivers:
            if column_value == overall_max:
                color = 'color:purple'
            elif column_value == max_values.get(driver):
                color = 'color:green'

        best_colors.append(color)

    return best_colors

tyres={
    'SOFT':'🔴',
    'MEDIUM':'🟡',
    'HARD':'⚪',
    'INTERMEDIATE':'🟢',
    'WET':'🔵',
    'SUPERSOFT': '🟣',
    'ULTRASOFT': '🟠',
    'HYPERSOFT': '🩷',
    'SUPERHARD': '🍊',
}

race_session = 'Practice 1'
if race_session == 'Practice 1':
    drivers_data = []
    driver_data_cols = [
        'Driver', 'Time',
        'Gap_ahead_Driver', 'Gap_to_Leader',
        'Sector1', 'I1', 'Sector2', 'I2', 'Sector3',
        'FL', 'LapTime', 'ST', 'Lap', 'Tyre', 'PitStop'
    ]
    fastest_lap_drivers = []
    for driver in session.drivers:
        fastest_lap_per_driver = session.laps.pick_drivers(driver).pick_fastest()
        try:
            if not fastest_lap_per_driver.empty:
                fastest_lap_per_driver = [
                    fastest_lap_per_driver.Driver  + ' ❚ ' + fastest_lap_per_driver.DriverNumber,
                    fastest_lap_per_driver.Time, 0, 0, fastest_lap_per_driver.Sector1Time, fastest_lap_per_driver.SpeedI1,
                    fastest_lap_per_driver.Sector2Time, fastest_lap_per_driver.SpeedI2, fastest_lap_per_driver.Sector3Time,
                    fastest_lap_per_driver.SpeedFL, fastest_lap_per_driver.LapTime, fastest_lap_per_driver.SpeedST,
                    fastest_lap_per_driver.LapNumber, fastest_lap_per_driver.Compound, (fastest_lap_per_driver.Stint - 1)
                ]
        except:
            continue
        fastest_lap_drivers.append(pd.Series(fastest_lap_per_driver, index=driver_data_cols))
        fastest_laps = pd.DataFrame(fastest_lap_drivers)
        
        fastest_laps['Tyre'] = fastest_laps.Tyre.replace(to_replace=tyres)
        fastest_laps = fastest_laps.sort_values(['LapTime'])
        fastest_laps['Gap_ahead_Driver'] = fastest_laps['LapTime'].diff()
        fastest_laps['Gap_to_Leader'] = fastest_laps['LapTime'] - fastest_laps['LapTime'].iloc[0]
        fastest_laps.index = range(1, len(fastest_laps) +1 )

    if fastest_laps.empty:
        print(f"No driver data collected for this scenario. Check session data and logic.")
    else:
        styled_df = fastest_laps.style
        styled_df = styled_df.apply(highlight_driver, subset=['Driver'])
        styled_df = styled_df.apply(is_personal_best_min, subset=['Sector1'], column = 'Sector1Time')
        styled_df = styled_df.apply(is_personal_best_min, subset=['Sector2'], column = 'Sector2Time')
        styled_df = styled_df.apply(is_personal_best_min, subset=['Sector3'], column = 'Sector3Time')
        styled_df = styled_df.apply(is_personal_best_min, subset=['LapTime'], column = 'LapTime')
        styled_df = styled_df.apply(is_personal_best_max, subset=['I1'], column = 'SpeedI1')
        styled_df = styled_df.apply(is_personal_best_max, subset=['I2'], column = 'SpeedI2')
        styled_df = styled_df.apply(is_personal_best_max, subset=['FL'], column = 'SpeedFL')
        styled_df = styled_df.apply(is_personal_best_max, subset=['ST'], column = 'SpeedST')
        styled_df = styled_df.apply(color_df, subset=['Gap_ahead_Driver'], color ='orange')
        styled_df = styled_df.apply(color_df, subset=['Gap_to_Leader'], color ='orange')

        formatters = {
        
        'Time': lambda x: str(x)[7:-3] if pd.notnull(x) else 'No Data',
        'Sector1': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
        'I1': lambda x: int(x) if pd.notnull(x) else 'No Data',
        'Sector2': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
        'I2': lambda x: int(x) if pd.notnull(x) else 'No Data',
        'Sector3': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
        'FL': lambda x: int(x) if pd.notnull(x) else 'No Data',
        'LapTime': lambda x: str(x)[11:-3] if pd.notnull(x) else 'No Data',
        'ST': lambda x: int(x) if pd.notnull(x) else 'No Data',
        'Lap': lambda x: int(x) if pd.notnull(x) else 'No Data',
        'PitStop': lambda x: int(x) if pd.notnull(x) else 'No Data',
        'Gap_ahead_Driver': lambda x: str(abs(x))[14:-3] if pd.notnull(x) else 'No Data',
        'Gap_to_Leader': lambda x: str(x)[14:-3] if pd.notnull(x) else 'No Data',
        }

        final_formatters = {k: v for k, v in formatters.items() if k in fastest_laps.columns}
        styled_df = styled_df.format(final_formatters)
styled_df

,Driver,Time,Gap_ahead_Driver,Gap_to_Leader,Sector1,I1,Sector2,I2,Sector3,FL,LapTime,ST,Lap,Tyre,PitStop
1,RUS ❚ 63,00:43:36.092,No Data,,16.683,313,29.005,244,19.854,283,1:05.542,311,13,🔴,2
2,VER ❚ 1,00:50:54.372,0.065,0.065,16.674,317,29.146,243,19.787,285,1:05.607,314,16,🔴,3
3,PIA ❚ 81,00:49:41.893,0.090,0.155,16.734,315,29.060,246,19.903,284,1:05.697,313,15,🔴,2
4,DUN ❚ 89,00:55:55.898,0.069,0.224,16.721,314,29.198,245,19.847,285,1:05.766,312,18,🔴,4
5,GAS ❚ 10,00:46:54.677,0.014,0.238,16.761,314,29.230,244,19.789,283,1:05.780,311,16,🔴,2
6,BOR ❚ 5,00:48:20.412,0.094,0.332,16.723,315,29.257,240,19.894,278,1:05.874,311,16,🔴,3
7,ALB ❚ 23,00:51:18.087,0.072,0.404,16.739,315,29.254,242,19.953,283,1:05.946,309,19,🔴,2
8,SAI ❚ 55,00:35:31.732,0.071,0.475,16.703,312,29.222,242,20.092,281,1:06.017,313,12,🟡,2
9,HAM ❚ 44,01:03:43.811,0.082,0.557,16.751,313,29.337,246,20.011,279,1:06.099,313,15,🔴,4
10,HAD ❚ 6,00:48:26.476,0.011,0.568,16.921,314,29.369,244,19.820,281,1:06.110,311,17,🔴,1


In [ ]:
Lap_Number=6

def is_personal_best(s, column):
    best_colors = []

    min_values = {}
    driver_ids = list(session.drivers)

    for driver_id in driver_ids:
        try:
            min_values[driver_id] = session.laps.pick_drivers(driver_id).pick_laps(range(0, Lap_Number + 1))[column].min()
        except Exception:
            min_values[driver_id] = float('inf')

    overall_min = float('inf')
    try:
        all_driver_laps = session.laps.pick_drivers(drivers_data_df.Driver).pick_laps(range(0, Lap_Number + 1))
        if not all_driver_laps.empty:
            overall_min = all_driver_laps[column].min()
    except Exception:
        overall_min = float('inf')

    for column_value in s:
        color = 'color:orange'
        for driver in session.drivers:
            if column_value == session.laps.pick_drivers(81).pick_laps(range(0, Lap_Number+1))[column].min():
                color = 'color:purple'
            if column_value == min_values.get(driver):
                color = 'color:green'

        if column_value < overall_min:
            color = 'color:purple'

        best_colors.append(color)

    return best_colors

def color_df(s, color):
    colors= []
    for driver_name in s:
        colors.append(f'color:{color}')
    return colors

def highlight_compound(s):
    colors = []
    for compound_value in s:
        if compound_value == 'SOFT':
            colors.append('color: red')
        elif compound_value == 'MEDIUM':
            colors.append('color: yellow')
        elif compound_value == 'HARD':
            colors.append('color: white')
        elif compound_value == 'INTERMEDIATE':
            colors.append('color: green')
        elif compound_value == 'WET':
            colors.append('color: blue')
        else:
            colors.append('color: grey')
    return colors

def highlight_driver(s):
    drivers = []
    for driver_name in s:
        if driver_name == 'VER' or driver_name == 'TSU':
            drivers.append('color: #0600ef')
        elif driver_name == 'GAS' or driver_name == 'DOO' or driver_name == 'COL':
            drivers.append('color: #ff87bc')
        elif driver_name == 'RUS' or driver_name == 'ANT':
            drivers.append('color: #27f4d2')
        elif driver_name == 'ALO' or driver_name == 'STR':
            drivers.append('color: #00665f')
        elif driver_name == 'LEC' or driver_name == 'HAM':
            drivers.append('color: #e80020')
        elif driver_name == 'ALB' or driver_name == 'SAI':
            drivers.append('color: #00a0dd')
        elif driver_name == 'HUL' or driver_name == 'BOR':
            drivers.append('color: #00e700')
        elif driver_name == 'HAD' or driver_name == 'LAW':
            drivers.append('color: #fcd700')
        elif driver_name == 'BEA' or driver_name == 'OCO':
            drivers.append('color: #b6babd')
        elif driver_name == 'NOR' or driver_name == 'PIA':
            drivers.append('color: #ff8000')
        else:
            drivers.append('color: grey')
    return drivers

drivers_data = []
driver_data_cols = [
    'Position', '№_Driver', 'Driver_Color', 'Driver', 'Time',
    'Gap_ahead_Driver', 'Gap_to_Leader',
    'Sector1', 'I1', 'Sector2', 'I2', 'Sector3',
    'FL', 'LapTime', 'ST', 'Lap', 'Compound', 'PitStop'
]

leader_driver_number = None
leader_lap_time_reference = None
leader_snapshot_lap_data = None

laps_at_snapshot = session.laps[session.laps['LapNumber'] == Lap_Number]

if not laps_at_snapshot.empty:
    leader_snapshot_lap_data = laps_at_snapshot.sort_values(by='Position').iloc[0]
    leader_driver_number = leader_snapshot_lap_data['DriverNumber']
    leader_lap_time_reference = leader_snapshot_lap_data['Time']
    try:
        leader_color = fastf1.plotting.get_driver_color(leader_snapshot_lap_data.Driver, session)
        leader_data = [
            leader_snapshot_lap_data.Position, leader_driver_number, leader_color, leader_snapshot_lap_data.Driver,
            leader_snapshot_lap_data.Time, 0, 0, leader_snapshot_lap_data.Sector1Time, leader_snapshot_lap_data.SpeedI1,
            leader_snapshot_lap_data.Sector2Time, leader_snapshot_lap_data.SpeedI2, leader_snapshot_lap_data.Sector3Time,
            leader_snapshot_lap_data.SpeedFL, leader_snapshot_lap_data.LapTime, leader_snapshot_lap_data.SpeedST,
            leader_snapshot_lap_data.LapNumber, leader_snapshot_lap_data.Compound, (leader_snapshot_lap_data.Stint - 1)
        ]
        drivers_data.append(pd.Series(leader_data, index=driver_data_cols))
    except Exception as e:
        print(f"Could not process leader's data for Lap {Lap_Number}. Error: {e}")
        exit()
else:
    print(f"No data available for Lap {Lap_Number} to determine the leader. Cannot proceed.")
    exit()


for driver_num in session.drivers:
    if driver_num == leader_driver_number:
        continue

    driver_laps = session.laps.pick_drivers(driver_num)
    potential_laps_after_leader = driver_laps[(driver_laps['Time'] >= leader_lap_time_reference)].copy()
    selected_lap = None
    if not potential_laps_after_leader.empty:
        selected_lap = potential_laps_after_leader.sort_values(by='LapNumber').iloc[0]
    else:
        fallback_lap = driver_laps[driver_laps['LapNumber'] == Lap_Number]
        if not fallback_lap.empty:
            selected_lap = fallback_lap.iloc[0]
        else:
            if not driver_laps.empty:
                selected_lap = driver_laps.iloc[-1]
            else:
                continue

    if selected_lap is not None and not selected_lap.empty:
        try:
            driver_color = fastf1.plotting.get_driver_color(selected_lap.Driver, session)
            driver_data = [
                selected_lap.Position, driver_num, driver_color, selected_lap.Driver,
                selected_lap.Time, 0, 0, selected_lap.Sector1Time, selected_lap.SpeedI1,
                selected_lap.Sector2Time, selected_lap.SpeedI2, selected_lap.Sector3Time,
                selected_lap.SpeedFL, selected_lap.LapTime, selected_lap.SpeedST,
                selected_lap.LapNumber, selected_lap.Compound, (selected_lap.Stint - 1)
            ]
            drivers_data.append(pd.Series(driver_data, index=driver_data_cols))
        except Exception as e:
            continue

drivers_data_df = pd.DataFrame(drivers_data)

if drivers_data_df.empty:
    print(f"No driver data collected for this scenario. Check session data and logic.")
else:
    drivers_data_df = drivers_data_df.fillna(0).infer_objects(copy=False)
    drivers_data_df = drivers_data_df.sort_values('Position').reset_index(drop=True)

    drivers_data_df['Time_td'] = drivers_data_df['Time']
    drivers_data_df['Gap_ahead_Driver'] = drivers_data_df['Time_td'].diff()
    drivers_data_df['Gap_to_Leader'] = drivers_data_df['Time_td'] - drivers_data_df['Time_td'].iloc[0]
   
    drivers_data_df = drivers_data_df.drop(columns=['Time_td', 'Driver_Color'])
    drivers_data_df = drivers_data_df[drivers_data_df['Position']>0]

    styled_df = drivers_data_df.style
    styled_df = styled_df.apply(highlight_driver, subset=['Driver'])
    styled_df = styled_df.apply(is_personal_best, subset=['Sector1'], column = 'Sector1Time')
    styled_df = styled_df.apply(is_personal_best, subset=['Sector2'], column = 'Sector2Time')
    styled_df = styled_df.apply(is_personal_best, subset=['Sector3'], column = 'Sector3Time')
    styled_df = styled_df.apply(is_personal_best, subset=['LapTime'], column = 'LapTime')
    styled_df = styled_df.apply(color_df, subset=['Gap_ahead_Driver'], color ='orange')
    styled_df = styled_df.apply(color_df, subset=['Gap_to_Leader'], color ='orange')
    styled_df = styled_df.apply(highlight_compound, subset=['Compound'])

    formatters = {
    
    'Position': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Time': lambda x: str(x)[7:-3] if pd.notnull(x) else 'No Data',
    'Sector1': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'I1': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Sector2': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'I2': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Sector3': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'FL': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'LapTime': lambda x: str(x)[11:-3] if pd.notnull(x) else 'No Data',
    'ST': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Lap': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'PitStop': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Gap_ahead_Driver': lambda x: str(abs(x))[7:-3] if pd.notnull(x) else 'No Data',
    'Gap_to_Leader': lambda x: str(x)[7:-3] if pd.notnull(x) else 'No Data',
    }

    final_formatters = {k: v for k, v in formatters.items() if k in drivers_data_df.columns}
    styled_df = styled_df.format(final_formatters)
styled_df
    

,Position,№_Driver,Driver,Time,Gap_ahead_Driver,Gap_to_Leader,Sector1,I1,Sector2,I2,Sector3,FL,LapTime,ST,Lap,Compound,PitStop


In [5]:
drivers_data = []
driver_data_cols = ['Time', 'Position', 'Driver_Number', 'Driver_Color', 'Driver_Name', 'Status']
for driver in session.drivers:
    driver_lap = session.laps.pick_drivers(driver)
    try:
        driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[Lap_Number], session)
        driver_data = [driver_lap.Time.iloc[Lap_Number], driver_lap.Position.iloc[Lap_Number], driver, driver_color, driver_lap.Driver.iloc[Lap_Number], driver_lap.Time.iloc[Lap_Number]]
    except:
        continue
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    drivers_data.append(driver_data_series)
drivers_data_df = pd.DataFrame(drivers_data)
drivers_data_df = drivers_data_df.fillna(0)
drivers_data_df.Time = drivers_data_df.Time.astype('str').str[6:-3]
drivers_data_df.Position = drivers_data_df.Position.astype('int')
drivers_data_df.Status = drivers_data_df.Status.astype('str').str[6:-3]
drivers_data_df

,Time,Position,Driver_Number,Driver_Color,Driver_Name,Status
0,00:49:16.999,0,4,#ff8000,NOR,00:49:16.999
1,00:43:48.852,0,16,#e80020,LEC,00:43:48.852
2,00:33:49.624,0,81,#ff8000,PIA,00:33:49.624
3,00:43:54.717,0,44,#e80020,HAM,00:43:54.717
4,00:48:48.206,0,63,#27f4d2,RUS,00:48:48.206
5,00:47:32.190,0,30,#fcd700,LAW,00:47:32.190
6,00:48:22.365,0,1,#0600ef,VER,00:48:22.365
7,00:34:32.330,0,5,#00e700,BOR,00:34:32.330
8,00:48:39.578,0,12,#27f4d2,ANT,00:48:39.578
9,00:34:05.152,0,10,#ff87bc,GAS,00:34:05.152


PER DRIVER

In [6]:
Lap_Number = 7
driver_data_last_laps = []

def highlight(s, min_max):
    if min_max == 'min':
        is_min_max = s == s.min()
    if min_max == 'max':
        is_min_max = s == s.min()

    return ['color: green' if v else 'color: orange' for v in is_min_max]

if Lap_Number<10:
    for Lap_Number in range(0, Lap_Number+1):
        driver_data_cols = ['Time', 'LapNumber', 'Sector1_Time', 'SpeedI1', 'Sector2_Time', 'SpeedI2', 'Sector3_Time', 'SpeedFL', 'LapTime', 'SpeedST']
        driver = 4
        driver_lap = session.laps.pick_drivers(driver)
        try:
            driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[Lap_Number], session)
            driver_data = [driver_lap.Time.iloc[Lap_Number], driver_lap.LapNumber.iloc[Lap_Number], driver_lap.Sector1Time.iloc[Lap_Number], driver_lap.SpeedI1.iloc[Lap_Number], driver_lap.Sector2Time.iloc[Lap_Number], driver_lap.SpeedI2.iloc[Lap_Number], driver_lap.Sector3Time.iloc[Lap_Number], driver_lap.SpeedFL.iloc[Lap_Number], driver_lap.LapTime.iloc[Lap_Number], driver_lap.SpeedST.iloc[Lap_Number]]
        except:
            continue
        driver_data_series = pd.Series(driver_data, index=driver_data_cols)
        driver_data_last_laps.append(driver_data_series)
    driver_data_last_laps_df = pd.DataFrame(driver_data_last_laps)
    styled_df = driver_data_last_laps_df.style
else:
    for Lap_Number in range(Lap_Number-9, Lap_Number+1):
        driver_data_cols = ['Time', 'LapNumber', 'Sector1_Time', 'SpeedI1', 'Sector2_Time', 'SpeedI2', 'Sector3_Time', 'SpeedFL', 'LapTime', 'SpeedST']
        driver = 4
        driver_lap = session.laps.pick_drivers(driver)
        try:
            driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[Lap_Number], session)
            driver_data = [driver_lap.Time.iloc[Lap_Number], driver_lap.LapNumber.iloc[Lap_Number], driver_lap.Sector1Time.iloc[Lap_Number], driver_lap.SpeedI1.iloc[Lap_Number], driver_lap.Sector2Time.iloc[Lap_Number], driver_lap.SpeedI2.iloc[Lap_Number], driver_lap.Sector3Time.iloc[Lap_Number], driver_lap.SpeedFL.iloc[Lap_Number], driver_lap.LapTime.iloc[Lap_Number], driver_lap.SpeedST.iloc[Lap_Number]]
        except:
            continue
        driver_data_series = pd.Series(driver_data, index=driver_data_cols)
        driver_data_last_laps.append(driver_data_series)
    driver_data_last_laps_df = pd.DataFrame(driver_data_last_laps)
    styled_df = driver_data_last_laps_df.style

styled_df = styled_df.apply(highlight, subset=['Sector1_Time'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['SpeedI1'], min_max='max')
styled_df = styled_df.apply(highlight, subset=['Sector2_Time'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['SpeedI2'], min_max='max')
styled_df = styled_df.apply(highlight, subset=['Sector3_Time'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['SpeedFL'], min_max='max')
styled_df = styled_df.apply(highlight, subset=['LapTime'], min_max='min')
styled_df = styled_df.apply(highlight, subset=['SpeedST'], min_max='max')

formatters = {
    'Time': lambda x: str(x)[7:-3] if pd.notnull(x) else 'No Data',
    'LapNumber': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Sector1_Time': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'SpeedI1': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Sector2_Time': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'SpeedI2': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'Sector3_Time': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'SpeedFL': lambda x: int(x) if pd.notnull(x) else 'No Data',
    'LapTime': lambda x: str(x)[11:-3] if pd.notnull(x) else 'No Data',
    'SpeedST': lambda x: int(x) if pd.notnull(x) else 'No Data',
}

final_formatters = {k: v for k, v in formatters.items() if k in driver_data_last_laps_df.columns}
styled_df = styled_df.format(final_formatters)

styled_df

,Time,LapNumber,Sector1_Time,SpeedI1,Sector2_Time,SpeedI2,Sector3_Time,SpeedFL,LapTime,SpeedST
0,00:23:14.776,1,No Data,271,38.857,149,24.841,286,No Data,273
1,00:24:19.448,2,16.434,316,28.843,250,19.395,288,1:04.672,318
2,00:25:53.967,3,27.082,184,37.232,208,30.205,No Data,1:34.519,221
3,00:32:14.682,4,No Data,217,38.018,172,23.724,287,No Data,271
4,00:33:19.559,5,16.407,314,28.763,250,19.707,281,1:04.877,318
5,00:35:07.173,6,19.034,207,42.308,73,46.272,No Data,1:47.614,193
6,00:49:16.999,7,No Data,267,43.927,205,22.067,288,No Data,293
7,00:50:21.409,8,16.424,313,28.623,250,19.363,287,1:04.410,321


In [7]:
def flag_color_row(s):
    flag_colors=[]
    for flag_color in s:
        match flag_color:
            case flag_color if 'GREEN' in flag_color:
                flag_colors.append('color:green')
            case flag_color if 'YELLOW' in flag_color:
                flag_colors.append('color:yellow')
            case flag_color if 'DOUBLE YELLOW' in flag_color:
                flag_colors.append('color:orange')
            case flag_color if 'RED' in flag_color:
                flag_colors.append('color:red')
            case flag_color if 'BLUE' in flag_color:
                flag_colors.append('color:blue')
            case flag_color if 'CLEAR' in flag_color:
                flag_colors.append('color:white')
            case flag_color if 'BLACK' in flag_color:
                flag_colors.append('color:dark-grey')
            case _:
                flag_colors.append('color:grey')
    return flag_colors

def color_df(s, color):
    colors= []
    for driver_name in s:
        colors.append(f'color:{color}')
    return colors


messages = fastf1.api.race_control_messages(session.api_path)
messages_df = pd.DataFrame.from_dict(messages)
messages_df['Message'] = messages_df.Flag.astype(str) + '/' + 'Lap ' + messages_df.Lap.astype(str) + ': ' + messages_df['Message']
messages_df = messages_df[messages_df['Lap'] <=Lap_Number]
messages_df= messages_df.drop(columns=['Category', 'Status', 'Scope', 'Sector', 'RacingNumber', 'Lap', 'Flag'])
messages_df = messages_df.fillna('None')
style_df = messages_df.style
style_df = style_df.apply(flag_color_row, subset=['Message'], axis=1)
style_df = style_df.apply(color_df, subset=['Time'], color='white')
style_df

req            INFO 	Using cached data for race_control_messages


,Time,Message


Sector1

In [8]:
def highlight_compound(s):
    colors = []
    for compound_value in s:
        if compound_value == 'SOFT':
            colors.append('color: red')
        elif compound_value == 'MEDIUM':
            colors.append('color: yellow')
        elif compound_value == 'HARD':
            colors.append('color: white')
        elif compound_value == 'INTERMEDIATE':
            colors.append('color: green')
        elif compound_value == 'WET':
            colors.append('color: blue')
        else:
            colors.append('color: grey')
    return colors

def color_df(s, color):
    colors= []
    for driver_name in s:
        colors.append(f'color:{color}')
    return colors

def highlight_driver(s):
    drivers = []
    for driver_name in s:
        driver_color = fastf1.plotting.get_driver_color(driver_name[0:3], session)
        drivers.append(f'color: {driver_color}')
    return drivers

tyres={
    'SOFT':'🔴',
    'MEDIUM':'🟡',
    'HARD':'⚪',
    'INTERMEDIATE':'🟢',
    'WET':'🔵',
    'SUPERSOFT': '🟣',
    'ULTRASOFT': '🟠',
    'HYPERSOFT': '🩷',
    'SUPERHARD': '🍊',
}

Lap_Number= 3

driver_data_last_laps_df=pd.DataFrame()
driver_data_best_laps = []
driver_data_cols = ['Driver', 'Sector1', 'Gap', 'Tyre']
for driver in session.drivers:
    
    driver_lap = session.laps.pick_drivers(driver).pick_laps(range(0, Lap_Number+1))
    try:
        driver_data = [driver_lap.Driver.iloc[0] + ' ❚ ' + driver_lap.DriverNumber.iloc[0], np.min(driver_lap.Sector1Time), 0, driver_lap.Compound.iloc[Lap_Number-1]]
    except:
        continue
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    driver_data_best_laps.append(driver_data_series)
driver_data_best_laps_df = pd.DataFrame(driver_data_best_laps)
driver_data_best_laps_df = driver_data_best_laps_df.sort_values('Sector1')
driver_data_best_laps_df['Tyre'] = driver_data_best_laps_df.Tyre.replace(to_replace=tyres)
driver_data_best_laps_df['Gap'] = ((driver_data_best_laps_df['Sector1'] - driver_data_best_laps_df['Sector1'].iloc[0])/driver_data_best_laps_df['Sector1'].iloc[0])*100
styled_df = driver_data_best_laps_df.style
styled_df = styled_df.apply(highlight_driver, subset=['Driver'])
styled_df = styled_df.apply(color_df, subset=['Sector1'], color ='orange')
styled_df = styled_df.apply(color_df, subset=['Gap'], color ='orange')
styled_df = styled_df.apply(highlight_compound, subset=['Tyre'])

formatters = {
    'Sector1': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'Gap': lambda x: str(round(x,1))+'%' if pd.notnull(x) else 'No Data',
}

final_formatters = {k: v for k, v in formatters.items() if k in driver_data_best_laps_df.columns}
styled_df = styled_df.format(final_formatters)
styled_df


,Driver,Sector1,Gap,Tyre
6,VER ❚ 1,16.370,0.0%,🔴
2,PIA ❚ 81,16.423,0.3%,🔴
0,NOR ❚ 4,16.434,0.4%,🔴
1,LEC ❚ 16,16.434,0.4%,🔴
4,RUS ❚ 63,16.447,0.5%,🔴
8,ANT ❚ 12,16.467,0.6%,🔴
17,TSU ❚ 22,16.527,1.0%,🔴
3,HAM ❚ 44,16.546,1.1%,🔴
15,STR ❚ 18,16.648,1.7%,🔴
7,BOR ❚ 5,16.650,1.7%,🔴


In [9]:
driver_data_last_laps_df=pd.DataFrame()
driver_data_best_laps = []
driver_data_cols = ['Color', 'Driver', 'Sector2', 'Compound']
for driver in session.drivers:
    try:
        driver_lap = session.laps.pick_drivers(driver).pick_laps(range(1, 53))
        driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[Lap_Number], session)
        driver_data = [driver_color, driver_lap.Driver.iloc[Lap_Number], np.min(driver_lap.Sector2Time), driver_lap.Compound.iloc[Lap_Number]]
    except:
        continue
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    driver_data_best_laps.append(driver_data_series)
driver_data_best_laps_df = pd.DataFrame(driver_data_best_laps)
styled_df = driver_data_best_laps_df.style
styled_df = styled_df.apply(highlight_compound, subset=['Compound'])

formatters = {
    'Sector2': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
}

final_formatters = {k: v for k, v in formatters.items() if k in driver_data_best_laps_df.columns}
styled_df = styled_df.format(final_formatters)
styled_df

,Color,Driver,Sector2,Compound
0,#ff8000,NOR,28.436,SOFT
1,#e80020,LEC,28.725,SOFT
2,#ff8000,PIA,28.694,SOFT
3,#e80020,HAM,28.703,SOFT
4,#27f4d2,RUS,28.979,SOFT
5,#fcd700,LAW,28.839,SOFT
6,#0600ef,VER,28.689,SOFT
7,#00e700,BOR,28.868,SOFT
8,#27f4d2,ANT,29.115,SOFT
9,#ff87bc,GAS,28.813,SOFT


In [10]:
def color_df(s, color):
    colors= []
    for driver_name in s:
        colors.append(f'color:{color}')
    return colors

driver_data_last_laps_df=pd.DataFrame()
driver_data_best_laps = []
driver_data_cols = ['Driver', 'Sector3', 'Gap_to_best', 'Compound']
for driver in session.drivers:
    try:
        driver_lap = session.laps.pick_drivers(driver).pick_laps(range(1, 53))
        driver_data = [driver_lap.Driver.iloc[Lap_Number], np.min(driver_lap.Sector1Time), 0, driver_lap.Compound.iloc[Lap_Number]]
    except:
        continue
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    driver_data_best_laps.append(driver_data_series)
driver_data_best_laps_df = pd.DataFrame(driver_data_best_laps)
driver_data_best_laps_df = driver_data_best_laps_df.sort_values('Sector3')
driver_data_best_laps_df['Gap_to_best'] = ((driver_data_best_laps_df['Sector3'] - driver_data_best_laps_df['Sector3'].iloc[0])/driver_data_best_laps_df['Sector3'].iloc[0])*100
styled_df = driver_data_best_laps_df.style
styled_df = styled_df.apply(highlight_driver, subset=['Driver'])
styled_df = styled_df.apply(color_df, subset=['Sector3'], color ='orange')
styled_df = styled_df.apply(color_df, subset=['Gap_to_best'], color ='orange')
styled_df = styled_df.apply(highlight_compound, subset=['Compound'])

formatters = {
    'Sector3': lambda x: str(x)[13:-3] if pd.notnull(x) else 'No Data',
    'Gap_to_best': lambda x: str(round(x,1))+'%' if pd.notnull(x) else 'No Data',
}

final_formatters = {k: v for k, v in formatters.items() if k in driver_data_best_laps_df.columns}
styled_df = styled_df.format(final_formatters)
styled_df

,Driver,Sector3,Gap_to_best,Compound
0,NOR,16.285,0.0%,SOFT
2,PIA,16.309,0.1%,SOFT
4,RUS,16.310,0.2%,SOFT
8,ANT,16.319,0.2%,SOFT
1,LEC,16.326,0.3%,SOFT
3,HAM,16.366,0.5%,SOFT
6,VER,16.370,0.5%,SOFT
11,ALB,16.395,0.7%,SOFT
9,GAS,16.461,1.1%,SOFT
18,SAI,16.464,1.1%,SOFT


In [11]:
driver_data_last_laps_df=pd.DataFrame()
driver_data_best_laps = []
driver_data_cols = ['Color', 'Driver', 'LapTime', 'Compound']
for driver in session.drivers:
    try:
        driver_lap = session.laps.pick_drivers(driver).pick_laps(range(1, 53))
        driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[Lap_Number], session)
        driver_data = [driver_color, driver_lap.Driver.iloc[Lap_Number], np.min(driver_lap.LapTime), driver_lap.Compound.iloc[Lap_Number]]
    except:
        continue
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    driver_data_best_laps.append(driver_data_series)
driver_data_best_laps_df = pd.DataFrame(driver_data_best_laps)
driver_data_best_laps_df = driver_data_best_laps_df.sort_values('LapTime')
driver_data_best_laps_df['GapBestLapTime'] = ((driver_data_best_laps_df.LapTime - driver_data_best_laps_df.LapTime.iloc[0])/driver_data_best_laps_df.LapTime.iloc[0])*100
driver_data_best_laps_df.LapTime = driver_data_best_laps_df.LapTime.astype(str).str[11:-3]
driver_data_best_laps_df.GapBestLapTime = driver_data_best_laps_df.GapBestLapTime.astype(str).str[:4]
driver_data_best_laps_df

,Color,Driver,LapTime,Compound,GapBestLapTime
0,#ff8000,NOR,1:03.971,SOFT,0.0
1,#e80020,LEC,1:04.492,SOFT,0.81
2,#ff8000,PIA,1:04.554,SOFT,0.91
3,#e80020,HAM,1:04.582,SOFT,0.95
4,#27f4d2,RUS,1:04.763,SOFT,1.23
6,#0600ef,VER,1:04.836,SOFT,1.35
7,#00e700,BOR,1:04.846,SOFT,1.36
9,#ff87bc,GAS,1:04.846,SOFT,1.36
5,#fcd700,LAW,1:04.926,SOFT,1.49
8,#27f4d2,ANT,1:05.052,SOFT,1.68


In [12]:
driver_data_last_laps_df=pd.DataFrame()
driver_data_best_laps = []
driver_data_cols = ['Color', 'Driver', 'TheoraticalBest', 'Compound']
for driver in session.drivers:
    try:
        driver_lap = session.laps.pick_drivers(driver).pick_laps(range(1, 53))
        driver_color = fastf1.plotting.get_driver_color(driver_lap.Driver.iloc[Lap_Number], session)
        driver_data = [driver_color, driver_lap.Driver.iloc[Lap_Number], (np.min(driver_lap.Sector1Time) + np.min(driver_lap.Sector2Time) + np.min(driver_lap.Sector3Time)), driver_lap.Compound.iloc[Lap_Number]]
    except:
        continue    
    driver_data_series = pd.Series(driver_data, index=driver_data_cols)
    driver_data_best_laps.append(driver_data_series)
driver_data_best_laps_df = pd.DataFrame(driver_data_best_laps)
driver_data_best_laps_df = driver_data_best_laps_df.sort_values('TheoraticalBest')
driver_data_best_laps_df['GapTheoraticalBest'] = round(((driver_data_best_laps_df.TheoraticalBest - driver_data_best_laps_df.TheoraticalBest.iloc[0])/driver_data_best_laps_df.TheoraticalBest.iloc[0]),4)*100
driver_data_best_laps_df.TheoraticalBest = driver_data_best_laps_df.TheoraticalBest.astype(str).str[11:-3]
driver_data_best_laps_df

,Color,Driver,TheoraticalBest,Compound,GapTheoraticalBest
0,#ff8000,NOR,1:03.971,SOFT,0.00
2,#ff8000,PIA,1:04.296,SOFT,0.51
1,#e80020,LEC,1:04.409,SOFT,0.68
3,#e80020,HAM,1:04.511,SOFT,0.84
6,#0600ef,VER,1:04.555,SOFT,0.91
4,#27f4d2,RUS,1:04.733,SOFT,1.19
5,#fcd700,LAW,1:04.835,SOFT,1.35
9,#ff87bc,GAS,1:04.846,SOFT,1.37
7,#00e700,BOR,1:04.846,SOFT,1.37
11,#00a0dd,ALB,1:04.909,SOFT,1.47


In [10]:
def background_color_df(s):
    colors= []
    for time_and_compound in s:
        if ' SOFT' in time_and_compound:
            colors.append('background-color:red')
        if ' MEDIUM' in time_and_compound:
            colors.append('background-color:yellow')
        if ' HARD' in time_and_compound:
            colors.append('background-color:white')
        if ' INTERMEDIATE' in time_and_compound:
            colors.append('background-color:green')
        if ' WET' in time_and_compound:
            colors.append('background-color:blue')
        if ' SUPERSOFT' in time_and_compound:
            colors.append('background-color:purple')
        if ' ULTRASOFT' in time_and_compound:
            colors.append('background-color:orangered')
        if ' HYPERSOFT' in time_and_compound:
            colors.append('background-color:pink')
        if ' SUPERHARD' in time_and_compound:
            colors.append('background-color:orange')
        if 'No data' in time_and_compound:
            colors.append('background-color:grey')
    return colors

Lap_Number = 5
driver_data_all_laps = []
if Lap_Number<10:
    lap_range =range(0, Lap_Number)
else :
    lap_range = range(Lap_Number-10, Lap_Number)
for lap in lap_range:
    LapTimePerLap = []
    driver_list = []
    for driver in session.drivers:
        try:
            driver_lap = session.laps.pick_drivers(driver).pick_laps(range(0, Lap_Number + 1))
            driver_data = str(driver_lap.LapTime.iloc[lap])  + ' ' + str(driver_lap.Compound.iloc[lap])
        except:
            driver_data = 'No data'
        if 'NaT' in driver_data:
            driver_data = 'No data'
        driver_list.append(driver_lap.Driver.iloc[0] + ' ❚ '+ driver)
        LapTimePerLap.append(driver_data)
    LapTimePerLapSeries = pd.Series(LapTimePerLap, index=driver_list)
    driver_data_all_laps.append(LapTimePerLapSeries)
driver_data_all_laps_df = pd.DataFrame(driver_data_all_laps)

driver_data_all_laps_df = driver_data_all_laps_df.fillna('No data')
if Lap_Number > 10 :
    driver_data_all_laps_df.index = range(Lap_Number-9, Lap_Number+1)
else:
    driver_data_all_laps_df.index = range(1, Lap_Number+1)

styled_df = driver_data_all_laps_df.style
styled_df.set_properties(**{'color': 'black'})
styled_df = styled_df.apply(background_color_df)

formatters = {}
for driver in driver_list:
    formatters[driver] = lambda x: (
        str(x).split(' ')[2][4:-3]
        if pd.notnull(x) and isinstance(x, str) and ' ' in str(x) and len(str(x).split(' ')) >= 4

        else 'No data'
    )

final_formatters = {k: v for k, v in formatters.items() if k in driver_data_all_laps_df.columns}
styled_df = styled_df.format(final_formatters)

header_styles = []
for i, col in enumerate(driver_data_all_laps_df.columns):
    driver_short_name = col.split(' ')[0]
    try:
        driver_color = fastf1.plotting.get_driver_color(driver_short_name, session)
    except Exception: 
        driver_color = '#000000' 
    header_styles.append({
        'selector': f'th.col_heading.col{i}',
        'props': [('color', driver_color)]
    })
styled_df = styled_df.set_table_styles(header_styles, overwrite=False)
styled_df

,NOR ❚ 4,LEC ❚ 16,PIA ❚ 81,HAM ❚ 44,RUS ❚ 63,LAW ❚ 30,VER ❚ 1,BOR ❚ 5,ANT ❚ 12,GAS ❚ 10,ALO ❚ 14,ALB ❚ 23,HAD ❚ 6,COL ❚ 43,BEA ❚ 87,STR ❚ 18,OCO ❚ 31,TSU ❚ 22,SAI ❚ 55,HUL ❚ 27
1,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data,No data
2,1:04.672,1:05.345,1:26.202,1:05.115,1:05.359,1:05.400,1:05.106,1:05.782,1:05.283,1:06.264,1:19.084,1:06.015,1:05.344,1:07.042,1:06.790,1:06.061,1:05.745,1:05.386,1:06.171,1:05.681
3,1:34.519,1:31.455,1:37.750,1:34.390,1:35.317,1:42.684,1:47.954,1:31.612,1:38.196,1:30.914,No data,1:58.087,1:26.792,1:25.094,1:33.590,1:31.244,1:36.148,1:31.668,1:44.239,1:49.836
4,No data,No data,1:05.004,No data,No data,No data,No data,No data,No data,No data,1:05.891,No data,No data,No data,No data,No data,No data,No data,2:24.144,No data
5,1:04.877,1:05.197,1:29.479,1:05.390,1:05.189,1:05.017,1:05.202,1:05.273,1:05.178,1:05.292,1:26.559,1:05.810,1:05.063,1:05.547,1:05.860,1:05.756,1:05.821,1:05.369,1:05.899,1:05.606
